In [1]:
import tatqa_utils
import pandas as pd

In [2]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_test.json')

In [8]:
devdf

,table,paragraphs,questions
0,{'uid': 'c4b92833-5c85-4bf4-b493-bc7741d759df'...,[{'uid': '04bfbe1d-235b-4036-95c2-e49983eb9cef...,[{'uid': '7c884c23-7774-4414-b817-d41dd797319b...
1,{'uid': '72342eec-d7d7-4ebd-ac45-5cf6cea20854'...,[{'uid': 'dec01fcd-5413-4c65-b6d9-bb133a964061...,[{'uid': 'f4137452-abf1-4a6b-a01b-52f9fda838b5...
2,{'uid': 'a66ccdd8-f680-45f5-958c-d086dbdb7a6f'...,[{'uid': '9d804915-eaa8-41ec-a46f-d115374f004e...,[{'uid': '070d313b-246b-4c42-9778-19c4423c4fcc...
3,{'uid': '88f81628-8b8b-4c7a-b0be-0fb1c0e3adbd'...,[{'uid': 'b36063e3-363f-4253-9b1c-482ef698012f...,[{'uid': 'ecda9a78-ff19-487e-953e-1827044fc0eb...
4,{'uid': '8c3c2602-da6e-4ff2-b9b5-fe720103aeed'...,[{'uid': '1cdf34e4-4681-47e8-94b0-2c8bf85b8dd8...,[{'uid': '90c1760e-25a7-42c9-86fa-cb4d674d9ae0...
...,...,...,...
273,{'uid': 'f093d959-5d51-4e09-9306-7e4dafe3e035'...,[{'uid': 'a7549455-bc82-4494-98fa-9e02bb0a129c...,[{'uid': '693401db-958c-4375-82de-44a9d21b385f...
274,{'uid': '8fc0b998-4631-426a-8006-7b2b4942768b'...,[{'uid': '444a3b80-d7b8-4bfd-8860-fcc111d0aa19...,[{'uid': 'b3ed5e20-92b9-4f99-9f06-956bfe669dba...
275,{'uid': '947aacd0-79b2-4daa-9a62-930c36896e10'...,[{'uid': '3dbbd54c-452c-43d0-8f1e-b43e9a0e464d...,[{'uid': '307dc920-4111-4823-bda8-7d3929196715...
276,{'uid': '1aaecafd-b535-41b6-9f50-66a7e29b30f7'...,[{'uid': 'd353e1ca-aecc-4fe0-a338-7fa5800ff623...,[{'uid': '61620b97-fea0-4a40-aa48-31ff0f42a880...


In [3]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

In [4]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [5]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [6]:
def gen_code(llm, question, table):
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    res = llm.invoke(prompt)
    code = res.content.replace('```python','').replace('```','')
    return code

def exec_code(code, table):  
    try: 
        loc = locals()   
        if not "run()" in code:
            exec(code + f"\nr = run({table})\n", globals(), loc)
        else: 
            exec(code + "\nr = run()\n", globals(), loc)
        return loc['r']
    except Exception as e:
            s = '[Error]'+ str(e)
            print(s)
            return (s,'')
#table = "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
#question = 'What is the change in Fixed Price in 2019 from 2018?'
table = "[['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]"
question = 'What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?'
code = gen_code(llm, question, table)
exec_code(code, table)

(14.065510597302506, '%')

In [7]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    #print (i)
    for q in item['questions']:        
        #if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from'] : 
        if True: 
            code = gen_code(llm, q, table)
            r = exec_code(code, table)
            err =  None
            if r is None:
                (pred_value, pred_unit) = (None, None)
            else:                
                if len(r) == 2:
                    (pred_value, pred_unit) = r                    
                    #print(pred_value, type(pred_value))
                    if isinstance(pred_value, str) and  pred_value.startswith('[Error]'):
                        (pred_value, pred_unit) = (None, None)
                        err,_ = r                                                        
                        
                elif len(r) == 1:
                    pred_value = r                
                else:
                    (pred_value, pred_unit) = (None, None)        
                
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_unit, q, code, item['table'], err))


0it [00:00, ?it/s]

KeyError: 'answer_type'

In [9]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=True)
print( pred_em, pred_f1, scale_score)

special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
0.6098514464425332 0.6340891321344798 0.6974198592650508


In [28]:
import pickle
with open('res_p8_unit.pickle', 'wb') as f:
    pickle.dump(res,f)

In [18]:
print([i for i in res if i[-1]  is not None][0])


({'answer_type': 'arithmetic', 'answer': 14.07, 'scale': 'percent'}, None, None, {'uid': 'a4f782f1-de3b-47a3-87af-1cb33c112b39', 'order': 6, 'question': 'What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?', 'answer': 14.07, 'derivation': '(592-519)/519', 'answer_type': 'arithmetic', 'answer_from': 'table', 'rel_paragraphs': [], 'req_comparison': False, 'scale': 'percent'}, "def run():\n    table = [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]\n    \n    value_2019 = int(table[5][1])\n    value_2018 = int(table[5][2])\n    \n    percentage_change = ((value_2019 - value_2

In [20]:
len((1,2))

2

In [32]:
res[0][-1]

In [16]:
def run():
    table = [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]
    
    value_2019 = int(table[5][1])
    value_2018 = int(table[5][2])
    
    percentage_change = ((value_2019 - value_2018) / value_2018) * 100
    
    return (round(percentage_change, 2), 'percent')
run()

ValueError: invalid literal for int() with base 10: '6,106'